# Table of Contents
 <p><div class="lev2"><a href="#Multilayer-Convolutional-Network"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Multilayer Convolutional Network</a></div><div class="lev2"><a href="#Convolution-and-Pooling"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Convolution and Pooling</a></div><div class="lev2"><a href="#First-Convolutional-Layer"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>First Convolutional Layer</a></div><div class="lev2"><a href="#second-layer"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>second layer</a></div><div class="lev2"><a href="#Densely-Connected-Layer"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Densely Connected Layer</a></div><div class="lev2"><a href="#dropout"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>dropout</a></div><div class="lev2"><a href="#readout-layer"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>readout layer</a></div><div class="lev2"><a href="#train-and-evaluate-the-model"><span class="toc-item-num">0.8&nbsp;&nbsp;</span>train and evaluate the model</a></div><div class="lev2"><a href="#最適化の方法について"><span class="toc-item-num">0.9&nbsp;&nbsp;</span>最適化の方法について</a></div><div class="lev3"><a href="#steepeset-gradient-descent-optimize"><span class="toc-item-num">0.9.1&nbsp;&nbsp;</span>steepeset gradient descent optimize</a></div><div class="lev3"><a href="#ADAM-optimzer"><span class="toc-item-num">0.9.2&nbsp;&nbsp;</span>ADAM optimzer</a></div><div class="lev2"><a href="#Sthochastic-Grdient-Descent(SGD)"><span class="toc-item-num">0.10&nbsp;&nbsp;</span>Sthochastic Grdient Descent(SGD)</a></div>

softmaxの復習

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())

In [4]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>

In [5]:
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [8]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
print(cross_entropy)
print(y)

0.9192
Tensor("Mean:0", shape=(), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)


## Multilayer Convolutional Network
ReLU neuron:

rectifier : 活性関数

$f(x)=\max(0,x)$

シグモイドやハイパボリックタンジェントよりも生物学的にも妥当で性能がいいとされる。この活性関数を使ったユニットをrectified linear unit(ReLU)と呼ぶ。

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling

In [12]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2,1], padding='SAME')

## First Convolutional Layer
畳み込みは5x5のパッチから32の特徴量を計算、最初の二次元がパッチサイズ、次がチャネル数、最後が出力数としてつくる

In [10]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

この層をあてるには、入力xを4次元のテンソルにする必要がある。2,3次元めが縦と横、4次元めは色のチャネル数、1次元めは？？

In [11]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

畳み込んで、バイアスを足して、ReLU関数をあて、max poolする

In [13]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [17]:
h_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [18]:
h_conv1

<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>

## second layer

イメージとしては32枚の特徴量画像（14x14ピクセル)に対してさらにもう一回畳み込みをする。なので3次元目(入力チャネル数)は32。そこから64チャネルつくる。

出力チャネル数は入力チャネルの定数倍じゃなくても大丈夫なの？->たぶん大丈夫。5x5x32に対していくつ出力するのかという話なだけだと思う。

In [31]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)  + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Densely Connected Layer

イメージサイズは7x7、64チャネルになった。

これを一列に並べて、全結合させる

In [32]:
W_fc1 = weight_variable([7 * 7 * 64, 1024]) 
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## dropout
オーバーフィッティングをさけるためにドロップアウトを適用する。

トレーニングのときはドロップアウトをオン、テストのときはオフにするために、placeholderを使う。

ドロップアウト(2014): コネクション、ユニットそれぞれ確率pでオフにする

In [22]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## readout layer

In [23]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## train and evaluate the model
ソフトマックスのときと違うこと：

1. steepeset gradient descent optimize -> ADAM optimzer
2. ドロップアウトのパラメーターを追加
3. ログ出力

In [36]:
accuracy

<tf.Tensor 'Mean_6:0' shape=() dtype=float32>

In [33]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(2000):
    batch  = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy) )
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

step 0, training accuracy 0.12
step 100, training accuracy 0.86
step 200, training accuracy 0.88
step 300, training accuracy 0.94
step 400, training accuracy 0.96
step 500, training accuracy 0.88
step 600, training accuracy 0.94
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 1
step 1100, training accuracy 0.98
step 1200, training accuracy 0.98
step 1300, training accuracy 0.96
step 1400, training accuracy 0.94
step 1500, training accuracy 0.94
step 1600, training accuracy 0.94
step 1700, training accuracy 0.92
step 1800, training accuracy 0.96
step 1900, training accuracy 1
test accuracy 0.9773


## 最適化の方法について
### steepeset gradient descent optimize 
基本的なアイデア：いまいる点にたいして導関数を計算して、それにプロポーショナルに移動すればよい

### ADAM optimzer

## Sthochastic Grdient Descent(SGD)
確率的降下法



https://ja.scribd.com/doc/260859670/30minutes-Adam